In [ ]:
# GPU supported version 
!pip install paddlepaddle-gpu==2.6.2 -f https://www.paddlepaddle.org.cn/whl/windows/mkl/avx/stable.html
!pip install paddleocr==2.9.1

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4
#after run of this cell restart runtime

In [ ]:
import paddle
paddle.device.is_compiled_with_cuda()

In [ ]:
import os
import json
import time
from paddleocr import PaddleOCR
from google.colab import drive
from tqdm import tqdm
import paddle

# Google Drive connection
drive.mount('/content/drive')

def is_image_file(filename):
    return filename.lower().endswith(('.png', '.jpg', '.jpeg'))

def run_paddleocr_on_existing_json(image_root, input_json_path, model_name='paddleocr'):
    """Adds PaddleOCR result into current JSON file """

    # OCR settings
    use_gpu = False
    try:
        if paddle.device.is_compiled_with_cuda():
            use_gpu = True
            paddle.device.set_device('gpu')
            print("GPU will use")
        else:
            print("cannot find GPU, CPU will be using")
    except Exception as e:
        print(f"CUDA control errror: {e}. CPU will use.")

    ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=use_gpu)

    # upload current JSON file
    with open(input_json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    start_time = time.time()

    # And change the loop to:
    for filename in tqdm([k for k in data if not k.startswith("_")], desc="OCR process"):
        
        for root, _, files in os.walk(image_root):
            if 'text_category' in root:
                continue
            if filename in files:
                img_path = os.path.join(root, filename)
                try:
                    result = ocr.ocr(img_path, cls=True)

                    extracted_text = []
                    if result and result[0]:
                        for line in result[0]:
                            extracted_text.append(line[1][0])

                    # write OCR result
                    data[filename]["models"][model_name] = {
                        "prediction": '\n'.join(extracted_text) if extracted_text else "",
                        "cer": None,
                        "wer": None
                    }
                except Exception as e:
                    print(f"{filename} error: {str(e)}")
                break  # if find file finish process

    elapsed = round(time.time() - start_time,2)
    print(f"\n OCR completed: {elapsed:.2f} second")

    meta = data.get("_meta", {})
    processing_times = meta.get("processing_times", {})
    processing_times[model_name] = elapsed
    meta["processing_times"] = processing_times
    data["_meta"] = meta



    
    with open(input_json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f" Saved updated JSON:  {input_json_path}")
    return data




input_json_path = '/content/drive/MyDrive/nutuk/benchmark/converted_data.json'
image_root = '/content/drive/MyDrive/nutuk/benchmark/'

#start ocr process
updated = run_paddleocr_on_existing_json(image_root, input_json_path)



